In [2]:
import sys
import os
import argparse
import random

SCRIPT_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Third-party
import numpy as np
import torch
import tensorflow as tf
from datasets import load_dataset
import ipynbname

# Communication
from basestation import User, EdgeServer
from user_association import uncertainty_aware_offloading
from utils import (
    estimate_workload,
    measure_inference_delay,
    generate_rayleigh_coeffs,
    bit_size_text,
)
from config import (
    BANDWIDTH,
    TRANSMIT_POWER,
    NOISE_POWER,
    LOCAL_COMPUTE_CAP,
    EDGE_COMPUTE_CAP,
    MAX_COMPUTE_PER_USER,
    SLM,
    LLM,
    K,
)

# LLM
from model.huggingface_model import HuggingfaceModel
from model import get_model

In [3]:
model = get_model(SLM)

In [4]:
dataset = load_dataset("Muennighoff/babi", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
print(dataset["passage"][0])
print(dataset["question"][0])

Mary moved to the bathroom.
John went to the hallway.

Where is Mary?


In [ ]:
passage = dataset["passage"][0]
question = dataset["question"][0]
answer = dataset["answer"][0]
input = f"Context: {passage.strip()}\n" f"Question: {question.strip()}\n" f"Answer: "
input

'Context: Mary moved to the bathroom.\nJohn went to the hallway.\nQuestion: Where is Mary?\nAnswer: '

In [12]:
output = model.generate(input)

In [13]:
topk = model.topk_probs(dataset["question"][0], 10)

In [14]:
topk

[('\n', 0.5563808083534241),
 ('', 0.10452341288328171),
 ('And', 0.06822796165943146),
 ('Now', 0.04549715295433998),
 ('The', 0.044508274644613266),
 ('A', 0.04053583741188049),
 ('You', 0.03975972905755043),
 ('When', 0.03753828629851341),
 ('Be', 0.03515499085187912),
 ('Remember', 0.027873560786247253)]

In [15]:
output

'Context: Mary moved to the bathroom.\nJohn went to the hallway.\nQuestion: Where is Mary?\nAnswer: 10 feet away.\nJohn went to the kitchen.\nQuestion: Where is the stove?\nAnswer: 10 feet away.\nJohn went to the living room.\nQuestion: Where is the TV?\nAnswer: '